In [2]:
import pandas as pd
import pandasql as ps

In [3]:
pathPedido = '../data/pedido.csv'
pathItem = '../data/item.csv'

In [4]:
pedido = pd.read_csv(pathPedido)

In [5]:
pedido.head()

,idPedido,idCliente,descSituacao,dtPedido,dtAprovado,dtEnvio,dtEntrega,dtEntregaEstimada
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [6]:
item = pd.read_csv(pathItem)

In [7]:
item.head()

,idPedido,idPedidoItem,idProduto,idVendedor,shipping_limit_date,vlPreco,vlFrete
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [8]:
tb_pedido_item = ps.sqldf(""" select
                                    t2.*,
                                    t1.dtPedido
                              from pedido as t1

                              left join item as t2
                              on t1.idPedido = t2.idPedido

                              WHERE t1.dtPedido < '2018-01-01'
                              and t1.dtPedido >= '2017-07-01'
                              and t2.idVendedor is not null
                """)

In [9]:
tb_pedido_item.head()

,idPedido,idPedidoItem,idProduto,idVendedor,shipping_limit_date,vlPreco,vlFrete,dtPedido
0,e481f51cbdc54678b7cc49136f2d6af7,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,2017-10-02 10:56:33
1,949d5b44dbf5de918fe9c16f97b45f8a,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20,2017-11-18 19:28:06
2,a4591c265e18cb1dcee52889e2d8acc3,1,060cb19345d90064d1015407193c233d,8581055ce74af1daba164fdbd55a40de,2017-07-13 22:10:13,147.90,27.36,2017-07-09 21:57:05
3,e69bfb5eb88e0ed6a785585b27e16dbf,1,9a78fb9862b10749a117f7fc3c31f051,7c67e1448b00f6e969d365cea6b010ab,2017-08-11 12:05:32,149.99,19.77,2017-07-29 11:55:02
4,34513ce0c4fab462a55830c0989c7edb,1,f7e0fa615b386bc9a8b9eb52bc1fff76,87142160b41353c4e5fca2360caf6f92,2017-07-19 20:10:08,98.00,16.13,2017-07-13 19:58:11


In [10]:
tb_summary = ps.sqldf("""
                    select idVendedor,
                    count(distinct idPedido) as qtdPedidos,
                    count(distinct date(dtPedido)) as qtdDias,
                    count(idProduto) as qtdItens,
                    (JulianDay('2018-01-01') - JulianDay(max(dtPedido))) as qtdRecencia,
                    sum(vlPreco) / count(distinct idPedido) as avgTicket,
                    avg(vlPreco) as avgValorProduto,
                    max(vlPreco) as maxValorProduto,
                    min(vlPreco) as minValorProduto,
                    count(idProduto) / count(distinct idPedido) as avgProdutoPedido
                    
                    from tb_pedido_item      
                    
                    group by idVendedor
""")

In [11]:
tb_summary.head()

,idVendedor,qtdPedidos,qtdDias,qtdItens,qtdRecencia,avgTicket,avgValorProduto,maxValorProduto,minValorProduto,avgProdutoPedido
0,0015a82c2db000af6aaaf3ae2ecb0532,3,3,3,74.655162,895.000000,895.000000,895.0,895.0,1
1,001cca7ae9ae17fb1caed9dfb1094831,98,68,116,4.034988,121.815816,102.913362,129.0,79.0,1
2,002100f778ceb8431b7a1020ff7ab48f,39,30,43,3.143391,20.505128,18.597674,49.9,9.9,1
3,003554e2dce176b5555353e4f3555ac8,1,1,1,16.713600,120.000000,120.000000,120.0,120.0,1
4,004c9cd9d87a3c30c522c48c4fc07416,84,66,87,8.097697,111.260476,107.423908,233.8,47.9,1


In [12]:
tb_pedido_summary = ps.sqldf("""
                        select 
                            idVendedor,
                            idPedido,
                            sum(vlPreco) as vlPreco
                            
                        from tb_pedido_item
                        
                        group by idVendedor, idPedido
                            

                """)

In [13]:
tb_pedido_summary.head()

,idVendedor,idPedido,vlPreco
0,0015a82c2db000af6aaaf3ae2ecb0532,7f39ba4c9052be115350065d07583cac,895.0
1,0015a82c2db000af6aaaf3ae2ecb0532,9dc8d1a6f16f1b89874c29c9d8d30447,895.0
2,0015a82c2db000af6aaaf3ae2ecb0532,d455a8cb295653b55abda06d434ab492,895.0
3,001cca7ae9ae17fb1caed9dfb1094831,038345c4a9d21c17180f2f77e1701c69,99.0
4,001cca7ae9ae17fb1caed9dfb1094831,06c07d5b2696b9631375e2877c955b3b,99.0


In [14]:
tb_min_max = ps.sqldf("""
                    select 
                        idVendedor,
                        max(vlPreco) as maxVlPedido,
                        min(vlPreco) as minVlPedido
                        
                    from tb_pedido_summary
                    
                    group by idVendedor
""")

In [15]:
tb_min_max.head()

,idVendedor,maxVlPedido,minVlPedido
0,0015a82c2db000af6aaaf3ae2ecb0532,895.0,895.0
1,001cca7ae9ae17fb1caed9dfb1094831,297.0,79.0
2,002100f778ceb8431b7a1020ff7ab48f,49.9,9.9
3,003554e2dce176b5555353e4f3555ac8,120.0,120.0
4,004c9cd9d87a3c30c522c48c4fc07416,233.8,47.9


In [16]:
tb_life = ps.sqldf(""" 
                              select
                                    t2.idVendedor,
                                    sum(vlPreco) as  LTV,
                                    ('2918-01-01' -max(dtPedido)) as qtdDiasBase
                              from pedido as t1

                              left join item as t2
                              on t1.idPedido = t2.idPedido

                              WHERE t1.dtPedido < '2018-01-01'
                              and t2.idVendedor is not null
                              
                              group by t2.idVendedor
                """)

In [17]:
tb_life.head()

,idVendedor,LTV,qtdDiasBase
0,0015a82c2db000af6aaaf3ae2ecb0532,2685.00,901
1,001cca7ae9ae17fb1caed9dfb1094831,21638.23,901
2,002100f778ceb8431b7a1020ff7ab48f,799.70,901
3,003554e2dce176b5555353e4f3555ac8,120.00,901
4,004c9cd9d87a3c30c522c48c4fc07416,16445.86,901


In [18]:
tb_dtPedido = ps.sqldf("""
                        select distinct
                            idVendedor,
                            date(dtPedido) as dtPedido,
                            lag(dtPedido) over(partition by idVendedor order by dtPedido) as lag1
                        
                        from tb_pedido_item                       
                        
                        order by idVendedor, dtPedido
""")

In [19]:
tb_dtPedido.head()

,idVendedor,dtPedido,lag1
0,0015a82c2db000af6aaaf3ae2ecb0532,2017-09-26,None
1,0015a82c2db000af6aaaf3ae2ecb0532,2017-10-12,2017-09-26 22:17:05
2,0015a82c2db000af6aaaf3ae2ecb0532,2017-10-18,2017-10-12 13:33:22
3,001cca7ae9ae17fb1caed9dfb1094831,2017-07-04,None
4,001cca7ae9ae17fb1caed9dfb1094831,2017-07-04,2017-07-04 11:49:17


In [20]:
tb_intervalo = ps.sqldf("""
                    select 
                        idVendedor,
                        avg(JulianDay(dtPedido) - (JulianDay(lag1))) as avgIntervaloVendas
                    
                    from tb_dtPedido
                    group by idVendedor
""")

In [21]:
tb_intervalo.head(30)

,idVendedor,avgIntervaloVendas
0,0015a82c2db000af6aaaf3ae2ecb0532,10.253316
1,001cca7ae9ae17fb1caed9dfb1094831,0.924744
2,002100f778ceb8431b7a1020ff7ab48f,1.892903
3,003554e2dce176b5555353e4f3555ac8,NaN
4,004c9cd9d87a3c30c522c48c4fc07416,1.413590
5,00ee68308b45bc5e2660cd833c3f81cc,1.645685
6,010543a62bd80aa422851e79a3bc7540,28.254086
7,013900e863eace745d3ec7614cab5b1a,14.706395
8,014c0679dd340a0e338872e7ec85666a,46.746476
9,014d9a685fd57276679edd00e07089e5,NaN


In [22]:
tb_teste = ps.sqldf("""
                        select
                            '2018-01-01' as dtReference,
                            t1.*,
                            t2.maxVlPedido,
                            t2.minVlPedido,
                            t3.LTV,
                            t3.qtdDiasBase,
                            t4.avgIntervaloVendas
                            
                            
                        from tb_summary as t1
                        
                        left join tb_min_max as t2
                        on t1.idVendedor = t2.idVendedor
                        
                        left join tb_life as t3
                        on t1.idVendedor = t3.idVendedor
                        
                        left join tb_intervalo as t4
                        on t1.idVendedor = t4.idVendedor
""")

In [23]:
tb_teste.head()

,dtReference,idVendedor,qtdPedidos,qtdDias,qtdItens,qtdRecencia,avgTicket,avgValorProduto,maxValorProduto,minValorProduto,avgProdutoPedido,maxVlPedido,minVlPedido,LTV,qtdDiasBase,avgIntervaloVendas
0,2018-01-01,0015a82c2db000af6aaaf3ae2ecb0532,3,3,3,74.655162,895.000000,895.000000,895.0,895.0,1,895.0,895.0,2685.00,901,10.253316
1,2018-01-01,001cca7ae9ae17fb1caed9dfb1094831,98,68,116,4.034988,121.815816,102.913362,129.0,79.0,1,297.0,79.0,21638.23,901,0.924744
2,2018-01-01,002100f778ceb8431b7a1020ff7ab48f,39,30,43,3.143391,20.505128,18.597674,49.9,9.9,1,49.9,9.9,799.70,901,1.892903
3,2018-01-01,003554e2dce176b5555353e4f3555ac8,1,1,1,16.713600,120.000000,120.000000,120.0,120.0,1,120.0,120.0,120.00,901,NaN
4,2018-01-01,004c9cd9d87a3c30c522c48c4fc07416,84,66,87,8.097697,111.260476,107.423908,233.8,47.9,1,233.8,47.9,16445.86,901,1.413590


In [24]:
pathVendedorVenda = '../data/fs_vendedor_venda.csv'

In [25]:
# tb_teste.to_csv(pathVendedorVenda, index = False, encoding = 'utf-8')